# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=91917fb99be5d230bdc234b22b6c9f65f684af6b099ceacbc6d07f88dbac9797
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark
You are working with 1 core(s)


In [29]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets/uw-madison-courses/'
os.listdir(path)

['courses.csv',
 'course_offerings.csv',
 'grade_distributions.csv',
 'rooms.csv',
 'instructors.csv',
 'schedules.csv',
 'sections.csv',
 'teachings.csv',
 'subjects.csv',
 'subject_memberships.csv']

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [4]:
df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'rooms', 'instructors', 'schedules', 'sections', 'teachings', 'subjects', 'subject_memberships']


Now check the contents of a few of the dataframses that were read in above.

In [5]:
# View the data
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [6]:
instructors.show(4,False)

+-------+------------------+
|id     |name              |
+-------+------------------+
|761703 |JOHN ARCHAMBAULT  |
|3677061|STEPHANIE KANN    |
|788586 |KATHY PREM        |
|1600463|KRISTIN KLARKOWSKI|
+-------+------------------+
only showing top 4 rows



## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [7]:
course_offerings.show()

+--------------------+--------------------+---------+--------------------+
|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
|f718e6cd-33f0-3c1...|a3e3e1c3-543d-3bb...|     1082|Cooperative Educa...|
|ea3b717c-d66b-30d...|a3e3e1c3-543d-3bb...|     1172|Cooperative Educa...|
|075da420-5f49-3dd...|a3e3e1c3-543d-3bb...|     1114|Cooperative Educa...|
|2b4e216d-a728-371...|a3e3e1c3-543d-3bb...|     1104|Cooperative Educa...|
|87966a7b-f676-33d...|a3e3e1c3-543d-3bb...|     1112|Cooperative Educa...|
|ff511882-5eab-3e7...|a3e3e1c3-543d-3bb...|     1134|Cooperative Educa...|
|b3e26604-fe6e-30d...|a3e3e1c3-543d-3bb...|     1084|Cooperative Educa...|
|e100d196-5e82-32e...|a3e3e1c3-543d-3bb...|     1154|Cooperative Educa...|
|76b9c458-d3c2-38c...|a3e3e1c3-543d-3bb...|     1162|Cooperative Educa...|
|2cd95315-cc90-33d...|a3e

In [8]:
sections.show()

+--------------------+--------------------+------------+------+--------------------+--------------------+
|                uuid|course_offering_uuid|section_type|number|           room_uuid|       schedule_uuid|
+--------------------+--------------------+------------+------+--------------------+--------------------+
|45adf63c-48c9-365...|344b3ebe-da7e-314...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|c6280e23-5e43-385...|f718e6cd-33f0-3c1...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|9395dc21-15d1-3fa...|ea3b717c-d66b-30d...|         FLD|     1|04368a56-c959-3e4...|f2d66a4d-0c08-3b4...|
|b99e440b-39db-350...|ea3b717c-d66b-30d...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|ca1c841f-41d5-329...|ea3b717c-d66b-30d...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|da41b0aa-2b81-378...|ea3b717c-d66b-30d...|         FLD|     1|04368a56-c959-3e4...|f2d66a4d-0c08-3b4...|
|51c4dc00-1fc7-3c7...|075da420-5f49-3dd...|   

In [9]:
rooms.show(5)

+--------------------+-------------+---------+
|                uuid|facility_code|room_code|
+--------------------+-------------+---------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|
|2cc50da3-ef0e-357...|         0032|     0249|
|ebbf62b4-2ac3-356...|         0032|     B101|
|ed828265-475b-31b...|         0032|     0549|
|b277dc8e-bab1-3a1...|         0032|     0349|
+--------------------+-------------+---------+
only showing top 5 rows



In [10]:
step1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how="left").select([rooms.uuid, rooms.facility_code, sections.course_offering_uuid, "number"])
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how="left").select([rooms.uuid, rooms.facility_code, "number", "term_code", "name"])
step2.limit(10).toPandas()


,uuid,facility_code,number,term_code,name
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1172,Cooperative Education Prog
1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1172,Cooperative Education Prog
2,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1114,Cooperative Education Prog
3,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1104,Cooperative Education Prog
4,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1134,Cooperative Education Prog
5,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1134,Cooperative Education Prog
6,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1084,Cooperative Education Prog
7,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1154,Cooperative Education Prog
8,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1154,Cooperative Education Prog
9,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,1,1162,Cooperative Education Prog


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [11]:
rooms.select('facility_code').distinct().collect()[:5]

[Row(facility_code='1404K'),
 Row(facility_code='0401'),
 Row(facility_code='0469'),
 Row(facility_code='0452'),
 Row(facility_code='0047')]

In [14]:
step2.where(step2.facility_code == '0469').show()

+--------------------+-------------+------+---------+--------------------+
|                uuid|facility_code|number|term_code|                name|
+--------------------+-------------+------+---------+--------------------+
|2da69d83-787a-3b0...|         0469|    18|     1182|Intro to Speech C...|
|2da69d83-787a-3b0...|         0469|    23|     1172|Intro to College ...|
|2da69d83-787a-3b0...|         0469|     2|     1172|Intro to College ...|
|2da69d83-787a-3b0...|         0469|     9|     1164|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    16|     1164|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    12|     1154|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    18|     1154|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    20|     1154|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    29|     1162|Intro to College ...|
|2da69d83-787a-3b0...|         0469|    25|     1162|Intro to College ...|
|2da69d83-787a-3b0...|   

In [16]:
step2.filter(step2.facility_code == "0469").limit(10).toPandas()

,uuid,facility_code,number,term_code,name
0,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1074,Fundamentals-High Strings
1,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1084,Fundamentals-High Strings
2,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1092,Fundamentals-High Strings
3,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1152,Fundamentals-Low Strings
4,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1092,Fundamentals-Low Strings
5,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1172,Fundamentals-Low Strings
6,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1172,Fundamentals-Low Strings
7,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1162,Fundamentals-Low Strings
8,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1162,Fundamentals-Low Strings
9,350c7321-2d4c-3485-8e0c-8005a6f5ac3d,0469,1,1142,Fundamentals-Low Strings


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [17]:
step1 = (subjects.join(subject_memberships, subjects.code == subject_memberships.subject_code,how="inner").select(["name", "course_offering_uuid"]).withColumnRenamed("name", "subject_name"))
step2 = step1.join(sections, step1.course_offering_uuid == sections.course_offering_uuid, how="left").select(["subject_name", "room_uuid"])
step3 = (step2.join(rooms, step2.room_uuid == rooms.uuid, how="left").select(["subject_name", "facility_code", "room_code"]))
step3.groupBy("facility_code", "subject_name").count().orderBy("facility_code").show(10, False) 


+-------------+---------------------------------+-----+
|facility_code|subject_name                     |count|
+-------------+---------------------------------+-----+
|null         |Cell and Regenerative Biology    |163  |
|null         |International Business           |28   |
|null         |Philosophy                       |483  |
|null         |Urban and Regional Planning      |225  |
|null         |Kinesiology                      |639  |
|null         |Asian American Studies           |25   |
|null         |Pharmaceutical Sciences          |712  |
|null         |Dairy Science                    |741  |
|null         |Art Education (Department of Art)|62   |
|null         |Nutritional Sciences             |661  |
+-------------+---------------------------------+-----+
only showing top 10 rows



In [18]:
subject_memberships.show(3)

+------------+--------------------+
|subject_code|course_offering_uuid|
+------------+--------------------+
|         220|344b3ebe-da7e-314...|
|         320|344b3ebe-da7e-314...|
|         346|344b3ebe-da7e-314...|
+------------+--------------------+
only showing top 3 rows



In [19]:
course_offerings.show(3)

+--------------------+--------------------+---------+--------------------+
|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
|f718e6cd-33f0-3c1...|a3e3e1c3-543d-3bb...|     1082|Cooperative Educa...|
|ea3b717c-d66b-30d...|a3e3e1c3-543d-3bb...|     1172|Cooperative Educa...|
+--------------------+--------------------+---------+--------------------+
only showing top 3 rows



In [20]:
sections.show(3)

+--------------------+--------------------+------------+------+--------------------+--------------------+
|                uuid|course_offering_uuid|section_type|number|           room_uuid|       schedule_uuid|
+--------------------+--------------------+------------+------+--------------------+--------------------+
|45adf63c-48c9-365...|344b3ebe-da7e-314...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|c6280e23-5e43-385...|f718e6cd-33f0-3c1...|         FLD|     1|                null|f2d66a4d-0c08-3b4...|
|9395dc21-15d1-3fa...|ea3b717c-d66b-30d...|         FLD|     1|04368a56-c959-3e4...|f2d66a4d-0c08-3b4...|
+--------------------+--------------------+------------+------+--------------------+--------------------+
only showing top 3 rows



In [21]:
subjects.show(5)

+----+--------------------+------------+
|code|                name|abbreviation|
+----+--------------------+------------+
| 908|        Soil Science|    SOIL SCI|
| 350|             ENGLISH|     ENGLISH|
| 351|English as a Seco...|         ESL|
| 230|Human Development...|        HDFS|
| 352|             English|        ENGL|
+----+--------------------+------------+
only showing top 5 rows



In [22]:
sub_course_facil = subjects.join(subject_memberships, subject_memberships.subject_code == subjects.code, how = 'inner').select(['subject_code', 'course_offering_uuid', 'abbreviation'])

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [24]:
course_offerings.show(10, False)


+------------------------------------+------------------------------------+---------+--------------------------+
|uuid                                |course_uuid                         |term_code|name                      |
+------------------------------------+------------------------------------+---------+--------------------------+
|344b3ebe-da7e-314c-83ed-9425269695fd|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1092     |Cooperative Education Prog|
|f718e6cd-33f0-3c14-a9a6-834d9c3610a8|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1082     |Cooperative Education Prog|
|ea3b717c-d66b-30dc-8b37-964d9688295f|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1172     |Cooperative Education Prog|
|075da420-5f49-3dd0-93df-13e3c152e1b1|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1114     |Cooperative Education Prog|
|2b4e216d-a728-3713-8c7c-19afffc6b2fd|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1104     |Cooperative Education Prog|
|87966a7b-f676-33d0-83d2-acdb67da6790|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1112     |Cooperative

In [25]:
grade_distributions.show(10, False)


+------------------------------------+--------------+-------+--------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+-------+--------+--------+-----------+
|course_offering_uuid                |section_number|a_count|ab_count|b_count|bc_count|c_count|d_count|f_count|s_count|u_count|cr_count|n_count|p_count|i_count|nw_count|nr_count|other_count|
+------------------------------------+--------------+-------+--------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+-------+--------+--------+-----------+
|344b3ebe-da7e-314c-83ed-9425269695fd|1             |105    |0       |0      |0       |0      |0      |0      |0      |0      |0       |0      |0      |0      |0       |0       |0          |
|f718e6cd-33f0-3c14-a9a6-834d9c3610a8|1             |158    |0       |0      |0       |0      |0      |0      |0      |0      |0       |0      |0      |1      |0       |0       |0          |
|ea3b717c-d66b-30dc-8b37-964d9688295f|1      

In [26]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step2 = (step1.join(course_offerings,step1.course_offering_uuid == course_offerings.uuid,how="left").select(["name", "sum(f_count)"]).orderBy("sum(f_count)"))
step2.toPandas().tail(5)

,name,sum(f_count)
80166,Calculus&Analytic Geometry,63
80167,Calculus&Analytic Geometry 1,64
80168,Calculus&Analytic Geometry,67
80169,Animal Biology,70
80170,Calc--Functns of Variables,72


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [27]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [30]:
chosen = spark.createDataFrame(["Monkey", "Elephant", "Hippopotamus"], StringType())

In [33]:
lev = zoo.join(chosen, levenshtein(zoo["Animal"], chosen["value"]) < 5, "left")
lev.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+



### Great job! 